
Importing DataSet from Drive


In [ ]:
!unzip /content/drive/MyDrive/NEGATIVE.zip  -d /content
!unzip /content/drive/MyDrive/POSITIVE.zip  -d /content
!unzip /content/drive/MyDrive/POSITIVEGAN.zip  -d /content/content/Licenta/Processed_RSNA2/

Streaming output truncated to the last 5000 lines.
  inflating: /content/content/Licenta/Processed_RSNA2/NEGATIVE/29087_951624387.png  
  inflating: /content/content/Licenta/Processed_RSNA2/NEGATIVE/28959_407797154.png  
  inflating: /content/content/Licenta/Processed_RSNA2/NEGATIVE/46144_112792357.png  
  inflating: /content/content/Licenta/Processed_RSNA2/NEGATIVE/21304_1273578482.png  
  inflating: /content/content/Licenta/Processed_RSNA2/NEGATIVE/2516_992751859.png  
  inflating: /content/content/Licenta/Processed_RSNA2/NEGATIVE/2182_2067189673.png  
  inflating: /content/content/Licenta/Processed_RSNA2/NEGATIVE/6685_2127534398.png  
  inflating: /content/content/Licenta/Processed_RSNA2/NEGATIVE/28914_1080307280.png  
  inflating: /content/content/Licenta/Processed_RSNA2/NEGATIVE/43539_765637762.png  
  inflating: /content/content/Licenta/Processed_RSNA2/NEGATIVE/44626_1988104905.png  
  inflating: /content/content/Licenta/Processed_RSNA2/NEGATIVE/11107_1843457717.png  
  inflating

Using ResNet18 model

In [ ]:
from tensorflow.keras.optimizers.legacy import Adam, SGD
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, Model
tf.config.optimizer.set_jit(True)
def resnet_block(input_tensor, filters, kernel_size=3, stride=1, conv_shortcut=True, name=None):
    x = layers.Conv2D(filters, kernel_size, strides=stride, padding="same",
                      kernel_initializer="he_normal", name=name + "_conv1")(input_tensor)
    x = layers.BatchNormalization(axis=3, name=name + "_bn1")(x)
    x = layers.Activation('relu', name=name + "_relu1")(x)

    x = layers.Conv2D(filters, kernel_size, padding="same",
                      kernel_initializer="he_normal", name=name + "_conv2")(x)
    x = layers.BatchNormalization(axis=3, name=name + "_bn2")(x)

    if conv_shortcut:
        shortcut = layers.Conv2D(filters, 1, strides=stride,
                                 kernel_initializer="he_normal", name=name + "_shortcut")(input_tensor)
        shortcut = layers.BatchNormalization(axis=3, name=name + "_shortcut_bn")(shortcut)
        x = layers.add([x, shortcut], name=name + "_add")
    else:
        x = layers.add([x, input_tensor], name=name + "_add")
    x = layers.Activation('relu', name=name + "_relu2")(x)
    return x

def resnet18(input_shape=(256, 256, 1)):
    img_input = layers.Input(shape=input_shape)

    x = layers.Conv2D(64, 7, strides=2, padding='same', kernel_initializer='he_normal', name='conv1')(img_input)
    x = layers.BatchNormalization(axis=3, name='bn_conv1')(x)
    x = layers.Activation('relu', name='relu_conv1')(x)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2), padding="same", name="pool1")(x)

    # Define the resnet blocks
    x = resnet_block(x, 64, stride=1, conv_shortcut=True, name="block1")
    x = resnet_block(x, 64, name="block2")
    x = resnet_block(x, 128, stride=2, conv_shortcut=True, name="block3")
    x = resnet_block(x, 128, name="block4")
    x = resnet_block(x, 256, stride=2, conv_shortcut=True, name="block5")
    x = resnet_block(x, 256, name="block6")
    x = resnet_block(x, 512, stride=2, conv_shortcut=True, name="block7")
    x = resnet_block(x, 512, name="block8")

    x = layers.GlobalAveragePooling2D(name="avg_pool")(x)

    # x = layers.Dropout(0.5)(x)

    x = layers.Dense(1, activation='sigmoid', name='fc1')(x)

    # Create model
    return Model(img_input, x, name='resnet18')

def create_model():
  model = resnet18(input_shape=(256, 256, 1))
  return model

Creating DataFrames

In [ ]:
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split, KFold

data_dir = '/content/content/Licenta/Processed_RSNA2/'

# Get list of all image file paths
image_paths = []
for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.endswith('.png'):
            image_paths.append(os.path.join(root, file))

# Generate labels from file paths
labels = [os.path.basename(os.path.dirname(path)) for path in image_paths]

# Split data into train_val and test sets
train_val_paths, test_paths, train_val_labels, test_labels = train_test_split(image_paths, labels, test_size=0.1)

# Create a DataFrame to store the file paths and labels for train_val and test sets
train_val_df = pd.DataFrame({
    'path': train_val_paths,
    'label': train_val_labels
})
test_df = pd.DataFrame({
    'path': test_paths,
    'label': test_labels
})
# Print number of images in each set after augmentation
print(f'Training set: {len(train_val_df)} images')
print(f'Test set: {len(test_df)} images')

Training set: 51909 images
Test set: 5768 images


In [ ]:
# Define the data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Define the batch size
batch_size = 64

# Create a test data generator (since test set does not change)
test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='path',
    y_col='label',
    target_size=(256, 256),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='binary'
)

kf = KFold(n_splits=5, shuffle=True)

# K-fold cross-validation loop
for fold, (train_idx, val_idx) in enumerate(kf.split(train_val_df)):
    print(f'Fold {fold + 1}')

    train_fold_df = train_val_df.iloc[train_idx].copy()
    val_fold_df = train_val_df.iloc[val_idx].copy()

    # Create data generators for training and validation
    train_generator = train_datagen.flow_from_dataframe(
        train_fold_df,
        x_col='path',
        y_col='label',
        target_size=(256, 256),
        color_mode='grayscale',
        batch_size=batch_size,
        class_mode='binary'
    )

    val_generator = test_datagen.flow_from_dataframe(
        val_fold_df,
        x_col='path',
        y_col='label',
        target_size=(256, 256),
        color_mode='grayscale',
        batch_size=batch_size,
        class_mode='binary'
    )

    checkpoint_path = '/content/drive/MyDrive/Checkpoint' + str(fold)

    checkpoint = ModelCheckpoint(checkpoint_path,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             save_weights_only=False,
                             mode='auto',
                             initial_value_threshold=None)


    early_stop = EarlyStopping(monitor='val_loss',
                           patience=10,
                           mode='min',
                           verbose=1)



    optimizer = Adam(learning_rate=0.0001)
    model = create_model()
    # model.summary()
    model.compile(optimizer=optimizer, loss='binary_crossentropy',
                metrics=[Precision(name='precision'), Recall(name='recall'), AUC(name='auc')])

    # Training the model
    model.fit(train_generator, validation_data=val_generator, epochs=3, steps_per_epoch=len(train_generator),
              validation_steps=len(val_generator), callbacks=[checkpoint, early_stop])

    # Save the model for this fold
    model.save(f'model_fold_{fold + 1}.h5')




Found 5768 validated image filenames belonging to 2 classes.
Fold 1
Found 41527 validated image filenames belonging to 2 classes.
Found 10382 validated image filenames belonging to 2 classes.
Epoch 1/3
649/649 [==============================] - ETA: 0s - loss: 0.1421 - precision: 0.8728 - recall: 0.5708 - auc: 0.8668
Epoch 1: val_loss improved from inf to 1.45483, saving model to /content/drive/MyDrive/Checkpoint0


649/649 [==============================] - 475s 681ms/step - loss: 0.1421 - precision: 0.8728 - recall: 0.5708 - auc: 0.8668 - val_loss: 1.4548 - val_precision: 0.1577 - val_recall: 0.8299 - val_auc: 0.8418
Epoch 2/3
649/649 [==============================] - ETA: 0s - loss: 0.1212 - precision: 0.9297 - recall: 0.6527 - auc: 0.8761
Epoch 2: val_loss improved from 1.45483 to 0.14986, saving model to /content/drive/MyDrive/Checkpoint0


649/649 [==============================] - 441s 680ms/step - loss: 0.1212 - precision: 0.9297 - recall: 0.6527 - auc: 0.8761 - val_loss: 0.1499 - val_precision: 0.9980 - val_recall: 0.6403 - val_auc: 0.8754
Epoch 3/3
649/649 [==============================] - ETA: 0s - loss: 0.1133 - precision: 0.9498 - recall: 0.6826 - auc: 0.8803
Epoch 3: val_loss improved from 0.14986 to 0.12004, saving model to /content/drive/MyDrive/Checkpoint0


649/649 [==============================] - 402s 619ms/step - loss: 0.1133 - precision: 0.9498 - recall: 0.6826 - auc: 0.8803 - val_loss: 0.1200 - val_precision: 0.9801 - val_recall: 0.7026 - val_auc: 0.8665
Fold 2
Found 41527 validated image filenames belonging to 2 classes.
Found 10382 validated image filenames belonging to 2 classes.
Epoch 1/3
649/649 [==============================] - ETA: 0s - loss: 0.1502 - precision: 0.8036 - recall: 0.5307 - auc: 0.8628
Epoch 1: val_loss improved from inf to 0.20084, saving model to /content/drive/MyDrive/Checkpoint1


649/649 [==============================] - 424s 629ms/step - loss: 0.1502 - precision: 0.8036 - recall: 0.5307 - auc: 0.8628 - val_loss: 0.2008 - val_precision: 0.6595 - val_recall: 0.6127 - val_auc: 0.8686
Epoch 2/3
649/649 [==============================] - ETA: 0s - loss: 0.1255 - precision: 0.9197 - recall: 0.6315 - auc: 0.8729
Epoch 2: val_loss improved from 0.20084 to 0.11657, saving model to /content/drive/MyDrive/Checkpoint1


...

649/649 [==============================] - 415s 616ms/step - loss: 0.1415 - precision: 0.8611 - recall: 0.5791 - auc: 0.8680 - val_loss: 0.1341 - val_precision: 0.9820 - val_recall: 0.6440 - val_auc: 0.8581
Epoch 2/3
649/649 [==============================] - ETA: 0s - loss: 0.1186 - precision: 0.9296 - recall: 0.6641 - auc: 0.8801
Epoch 2: val_loss improved from 0.13409 to 0.11189, saving model to /content/drive/MyDrive/Checkpoint2


649/649 [==============================] - 419s 646ms/step - loss: 0.1186 - precision: 0.9296 - recall: 0.6641 - auc: 0.8801 - val_loss: 0.1119 - val_precision: 0.9942 - val_recall: 0.6754 - val_auc: 0.8730
Epoch 3/3
649/649 [==============================] - ETA: 0s - loss: 0.1138 - precision: 0.9417 - recall: 0.6864 - auc: 0.8849
Epoch 3: val_loss did not improve from 0.11189
649/649 [==============================] - 401s 618ms/step - loss: 0.1138 - precision: 0.9417 - recall: 0.6864 - auc: 0.8849 - val_loss: 0.2835 - val_precision: 1.0000 - val_recall: 0.0785 - val_auc: 0.7529
Fold 4
Found 41527 validated image filenames belonging to 2 classes.
Found 10382 validated image filenames belonging to 2 classes.
Epoch 1/3
649/649 [==============================] - ETA: 0s - loss: 0.1476 - precision: 0.8269 - recall: 0.5388 - auc: 0.8714
Epoch 1: val_loss improved from inf to 0.11517, saving model to /content/drive/MyDrive/Checkpoint3


649/649 [==============================] - 424s 630ms/step - loss: 0.1476 - precision: 0.8269 - recall: 0.5388 - auc: 0.8714 - val_loss: 0.1152 - val_precision: 0.9726 - val_recall: 0.7056 - val_auc: 0.8723
Epoch 2/3
649/649 [==============================] - ETA: 0s - loss: 0.1273 - precision: 0.9013 - recall: 0.6283 - auc: 0.8737
Epoch 2: val_loss did not improve from 0.11517
649/649 [==============================] - 416s 641ms/step - loss: 0.1273 - precision: 0.9013 - recall: 0.6283 - auc: 0.8737 - val_loss: 0.1959 - val_precision: 1.0000 - val_recall: 0.0875 - val_auc: 0.8640
Epoch 3/3
649/649 [==============================] - ETA: 0s - loss: 0.1194 - precision: 0.9392 - recall: 0.6593 - auc: 0.8803
Epoch 3: val_loss improved from 0.11517 to 0.10193, saving model to /content/drive/MyDrive/Checkpoint3


649/649 [==============================] - 413s 636ms/step - loss: 0.1194 - precision: 0.9392 - recall: 0.6593 - auc: 0.8803 - val_loss: 0.1019 - val_precision: 0.9870 - val_recall: 0.7056 - val_auc: 0.8794
Fold 5
Found 41528 validated image filenames belonging to 2 classes.
Found 10381 validated image filenames belonging to 2 classes.
Epoch 1/3
649/649 [==============================] - ETA: 0s - loss: 0.1447 - precision: 0.8633 - recall: 0.5600 - auc: 0.8674
Epoch 1: val_loss improved from inf to 0.10649, saving model to /content/drive/MyDrive/Checkpoint4


649/649 [==============================] - 437s 647ms/step - loss: 0.1447 - precision: 0.8633 - recall: 0.5600 - auc: 0.8674 - val_loss: 0.1065 - val_precision: 0.9638 - val_recall: 0.7384 - val_auc: 0.8818
Epoch 2/3
649/649 [==============================] - ETA: 0s - loss: 0.1242 - precision: 0.9304 - recall: 0.6396 - auc: 0.8741
Epoch 2: val_loss did not improve from 0.10649
649/649 [==============================] - 428s 660ms/step - loss: 0.1242 - precision: 0.9304 - recall: 0.6396 - auc: 0.8741 - val_loss: 0.1131 - val_precision: 0.9590 - val_recall: 0.7424 - val_auc: 0.8930
Epoch 3/3
649/649 [==============================] - ETA: 0s - loss: 0.1196 - precision: 0.9375 - recall: 0.6551 - auc: 0.8789
Epoch 3: val_loss did not improve from 0.10649
649/649 [==============================] - 413s 636ms/step - loss: 0.1196 - precision: 0.9375 - recall: 0.6551 - auc: 0.8789 - val_loss: 0.1975 - val_precision: 0.9908 - val_recall: 0.1427 - val_auc: 0.8128


In [ ]:
!cp /content/model_fold_2.h5 /content/drive/MyDrive/ModeleBest/
!cp /content/model_fold_3.h5 /content/drive/MyDrive/ModeleBest/
!cp /content/model_fold_4.h5 /content/drive/MyDrive/ModeleBest/
!cp /content/model_fold_5.h5 /content/drive/MyDrive/ModeleBest/

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy, TruePositives, TrueNegatives, FalsePositives, FalseNegatives
from tqdm import tqdm
import numpy as np

pre = Precision()
re = Recall()
acc = BinaryAccuracy()
tp = TruePositives()
tn = TrueNegatives()
fp = FalsePositives()
fn = FalseNegatives()

# Prepare your models
models = [load_model(f'model_fold_{i}.h5') for i in range(1, 6)]

for i, batch in tqdm(enumerate(test_generator), total=len(test_generator)):
    X, y = batch
    predictions = [model.predict(X) for model in models]
    # Use the np.round() function to get the binary predictions and then use np.mean() along axis=0 to compute the average
    # After that use np.round() again to get the final prediction, 0 or 1
    yhat = np.round(np.mean([np.round(pred) for pred in predictions], axis=0))

    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)
    tp.update_state(y, yhat)
    tn.update_state(y, yhat)
    fp.update_state(y, yhat)
    fn.update_state(y, yhat)
    if(i>len(test_generator)):
        break

print("Precision:", pre.result().numpy())
print("Recall:", re.result().numpy())
print("Binary Accuracy:", acc.result().numpy())
print("True Positives:", tp.result().numpy())
print("True Negatives:", tn.result().numpy())
print("False Positives:", fp.result().numpy())
print("False Negatives:", fn.result().numpy())


  0%|          | 0/91 [00:00<?, ?it/s]

2/2 [==============================] - 1s 49ms/step


  1%|          | 1/91 [00:10<15:45, 10.50s/it]

2/2 [==============================] - 0s 36ms/step


  2%|▏         | 2/91 [00:11<07:23,  4.98s/it]

...

2/2 [==============================] - 0s 31ms/step


92it [02:02,  1.76s/it]                        

2/2 [==============================] - 0s 31ms/step


92it [02:03,  1.34s/it]

Precision: 0.995283
Recall: 0.55526316
Binary Accuracy: 0.9711669
True Positives: 211.0
True Negatives: 5515.0
False Positives: 1.0
False Negatives: 169.0


In [ ]:
!cp /content/model_fold_1.h5 /content/drive/MyDrive/ModeleBest/
